In [2]:
import sys
import os
sys.path.append(os.path.abspath('../src')) # include top level package in python path

In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
.container { 
    width: 100% !important;
}
</style>
"""))

In [4]:
import torch
from torch import nn, Tensor
from model.fuzzy_logic import ProductLogic, MinimumLogic, LukasiewiczLogic, DrasticLogic, SchweizerSklarLogic
from model.fuzzy_layer import FuzzyMLP, FuzzyParam
from model.bool_logic import BoolLogic
from cache import TestMetric, TrainingRegime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

device = 'cuda'

logic = BoolLogic()
flogic = ProductLogic()

In [40]:
print(torch.__version__)
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(
    torch.tensor([[0.,1.0,0.]]), 
    torch.tensor([[0.1,0.7,0.2]])
)
loss

1.10.2


tensor([0.2689, 0.7311])

In [4]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

trainset = datasets.MNIST(root='./_mnist', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.MNIST(root='./_mnist', train=False, download=True, transform=transforms.ToTensor())

trainloader = DataLoader(trainset, batch_size=128)
testloader = DataLoader(testset, batch_size=128)

In [5]:
class CorrectOutputMetric(TestMetric):
    def name(self):
        return "correct-output"
    
    def measure_model(self, model, it):
        correct_count = 0
        total_count = 0

        with torch.no_grad():
            for imgs, labels in it():
                preds = model(imgs.to(device))
                if hasattr(model.__class__, "zero_reg"):
                    model.zero_reg()
                pred_labels = preds.argmax(dim=-1)
                correct_count += (pred_labels == labels.to(device)).sum()
                total_count += labels.numel()

        return (correct_count / total_count).item()
    
class LossMetric(TestMetric):
    def name(self):
        return "test-loss"

    def measure_model(self, model, it):
        loss_fn = nn.CrossEntropyLoss()

        total_loss = 0
        total_count = 0
        
        with torch.no_grad():
            for imgs, labels in it():
                preds = model(imgs.to(device))
                if hasattr(model.__class__, "zero_reg"):
                    model.zero_reg()
                loss = loss_fn(preds, labels.to(device))
                total_loss += loss
                total_count += labels.numel()

        return (total_loss / total_count).item()   
    
class UnsatisfiedMetric(TestMetric):
    def name(self):
        return "unsatisfied"

    def measure_model(self, model, it):
        total = 0
        
        for imgs, _ in it():
            total += model.from_all_layers(
                imgs.to(device),
                lambda lyr, inp, outp: (
                    1 - lyr.satisfactions(inp, outp)
                ).sum()
            )

        return total.item()
    
class ExampleMetric(TestMetric):
    def name(self):
        return "examples"

    def measure_model(self, model, it):
        total = 0
        
        for imgs, _ in it():
            total += model.from_all_layers(
                imgs.to(device),
                lambda lyr, inp, outp: (
                    lyr.examples(inp, outp)
                ).sum()
            )

        return total.item()
    
class CrispnessMetric(TestMetric):
    def name(self):
        return "crispness"

    def measure_model(self, model, it):
        total = 0
        
        for imgs, _ in it():
            total += model.from_all_layers(
                imgs.to(device),
                lambda lyr, inp, outp: (
                    lyr.crispness()
                ).sum()
            )

        return total.item()
    
class CrispConjunctionWidthsMetric(TestMetric):
    def name(self):
        return "crisp-widths"

    def measure_model(self, model, it):
        total_width = 0
        total_vars = 0
        
        with torch.no_grad():
            for layer in model.layers:
                values = layer.fuzzy_m.crisp_value()
                total_width += values.sum()
                total_vars += values.numel()

        return (total_width / total_vars).item()
    
class PreconditionsMetric(TestMetric):
    def name(self):
        return "preconditions"

    def measure_model(self, model, it):
        total = 0
        
        for imgs, _ in it():
            total += model.from_all_layers(
                imgs.to(device),
                lambda lyr, inp, outp: (
                    lyr.preconditions(inp)
                ).sum()
            )

        return total.item()
    
class ConsequentMetric(TestMetric):
    def name(self):
        return "consequent"

    def measure_model(self, model, it):
        total = 0
        
        for imgs, _ in it():
            total += model.from_all_layers(
                imgs.to(device),
                lambda lyr, inp, outp: (
                    outp.sigmoid()
                ).sum()
            )

        return total.item() * model.clauses_per_output
    
class TotalMetric(TestMetric):
    def name(self):
        return "total"

    def measure_model(self, model, it):
        total_vars = 0
        length = len(testset)
        
        with torch.no_grad():
            for layer in model.layers:
                values = layer.fuzzy_m.crisp_value()
                total_vars += values.numel() / values.size(dim=-1)
                
        return total_vars * length

In [6]:
exp = 2

class FuzzyMNIST(nn.Module):
    
    def __init__(self, logic, layers: tuple[tuple[int,...],...], clauses_per_output: int, signed_clauses: bool, maxsat: bool):
        super().__init__()
        self.logic = logic
        self.clauses_per_output = clauses_per_output
        self.signed_clauses = signed_clauses
        self.maxsat = maxsat
        
        self.layers = nn.ModuleList([
            FuzzyMLP(logic, shape, clauses_per_output, signed_clauses) for shape in layers
        ])
        
        self.logic_loss = nn.CrossEntropyLoss()
        
        #self.reg = None
        #self.pos = None
        #self.crisp = None
        self.diff = None
        
    def forward(self, input):
        prev_output = input.flatten(start_dim=1)
        #self.reg = torch.tensor(0.0, device=input.device)
        #self.pos = torch.tensor(0.0, device=input.device)
        #self.crisp = torch.tensor(0.0, device=input.device)
        self.diff = torch.tensor(0.0, device=input.device)
        for idx, layer in enumerate(self.layers):
            output = layer(prev_output)
            if idx > 0:
                #exs = layer.satisfactions(prev_output, output)
                #self.reg += ((1 - exs) ** exp).sum()
                #pos_exs = layer.examples(prev_output, output)
                #self.pos += (pos_exs ** exp).sum()
                #crisp_exs = layer.crispness()
                #self.crisp += (crisp_exs ** 1).sum()
                #if self.maxsat:
                #    precs = layer.preconditions(prev_output.detach())
                #    self.diff += (self.logic.implies(precs, output.unsqueeze(-1).detach()) ** exp).sum()
                #    self.diff += 10 * (self.logic.bin_conjoin(precs, output.unsqueeze(-1).detach()) ** exp).sum()
                #else:
                #    preds = layer.logic_preds(prev_output.detach())
                #    self.diff += (self.logic.implies(preds, output.detach()) ** exp).sum()
                #    self.diff += 10 * (self.logic.bin_conjoin(preds, output.detach()) ** exp).sum()
                preds = layer.logic_preds(prev_output.detach())
                output_probs = torch.softmax(output.detach(), dim=-1)
                self.diff += self.logic_loss(preds, output_probs)
            prev_output = output
            
        return torch.softmax(prev_output, dim=-1)
    
    def from_all_layers(self, input, f):
        total = torch.tensor(0.0, device=input.device)
        
        with torch.no_grad():
            prev_output = input.flatten(start_dim=1)
            for idx, layer in enumerate(self.layers):
                output = layer(prev_output)
                if idx > 0:
                    total += f(layer, prev_output, output).sum()
                prev_output = output
                
        return total
    
    def logic_reg(self):
        return self.reg
    
    def pos_reg(self):
        return self.pos
    
    def crisp_reg(self):
        return self.crisp
    
    def diff_reg(self):
        return self.diff
    
    def zero_reg(self):
        #self.reg = None
        #self.pos = None
        #self.crisp = None
        self.diff = None

In [7]:
class MNISTRealRegime(TrainingRegime):
    def __init__(
        self, 
        logic, 
        layers, 
        clauses_per_output,
        diff_weight,
        maxsat,
        #signed_clauses, 
        #reg_weight, 
        #pos_weight, 
        #crisp_weight, 
        lr=1e-2, 
        no_runs=1
    ):
        super().__init__("./mnist/", no_runs)
        
        self.flogic = logic
        self.layers = layers
        self.clauses_per_output = clauses_per_output
        self.diff_weight = diff_weight
        #self.maxsat = maxsat
        #self.signed_clauses = signed_clauses
        #self.reg_weight = reg_weight
        #self.pos_weight = pos_weight
        #self.crisp_weight = crisp_weight
        self.lr = lr
        
        self.tests = [
            CorrectOutputMetric(),
            LossMetric(),
            UnsatisfiedMetric(),
            ExampleMetric(),
            CrispnessMetric(),
            CrispConjunctionWidthsMetric(),
            PreconditionsMetric(),
            ConsequentMetric(),
            TotalMetric(),
        ]
        
        self.trainloader = DataLoader(trainset, batch_size=128)
        self.testloader = DataLoader(testset, batch_size=128)
        
        self.optims = [None] * no_runs
        
    def get_logic_str(self):
        if isinstance(self.flogic, ProductLogic):
            return "product"
        elif isinstance(self.flogic, MinimumLogic):
            return "minimum"
        elif isinstance(self.flogic, LukasiewiczLogic):
            return "lukasiewicz"
        elif isinstance(self.flogic, DrasticLogic):
            return "drastic"
        elif isinstance(self.flogic, SchweizerSklarLogic):
            return "ss" 
        else:
            return "fuzzy"
        
    def get_optim(self, run_no):
        optim = self.optims[run_no - 1]
        if optim is None:
            model = self.get_loaded_model(run_no)
            optim = self.optims[run_no - 1] = (
                torch.optim.Adam(model.parameters(), lr=self.lr)
            )
        return optim
        
    def regime_str(self):
        layer_str = " ".join(["(" + ",".join(str(i) for i in shape) + ")" for shape in self.layers])
        #signed_str = "SIGNED " if self.signed_clauses else ""
        maxsat_str = "| MAXSAT " if self.maxsat else ""
        return (
            "MNIST | ADAM, LR = %s | %s LOGIC | LAYERS = %s | DIFF WEIGHT %s %s| %s CLAUSES"
            % ( 
                self.lr,
                self.get_logic_str().upper(), 
                layer_str, 
                #self.reg_weight,
                #self.pos_weight,
                #self.crisp_weight,
                self.diff_weight,
                maxsat_str,
                self.clauses_per_output,
                #signed_str,
            )
        ) 
        
    def regime_filename_elems(self):
        layer_strs = ["l" + "-".join(str(i) for i in shape) for shape in self.layers]
        elems = [
            "mnistjoint",
            str(self.lr),
            self.get_logic_str(),
            *layer_strs,
            #"w%s" % self.reg_weight,
            #"p%s" % self.pos_weight,
            #"cr%s" % self.crisp_weight,
            "d%s" % self.diff_weight,
            "c%s" % self.clauses_per_output
        ]
        if self.maxsat:
            elems.append("max")
        return elems
        
    def training_dataloader(self, run_no):
        return self.trainloader
    
    def testing_dataloader(self, run_no):
        return self.testloader
    
    def training_step(self, run_no, model):
        optim = self.get_optim(run_no)
        loss_fn = nn.CrossEntropyLoss()
        
        def print_grad(grad):
            print(grad)
            return grad
        
        def step(tup):
            imgs, labels = tup
            preds = model(imgs.to(device))
            loss = loss_fn(preds, labels.to(device))
            #reg = model.logic_reg()
            #pos = model.pos_reg()
            #crisp = model.crisp_reg()
            diff = model.diff_reg()
            #if self.reg_weight != 0:
            #    loss = loss + self.reg_weight * reg
            #if self.pos_weight != 0:
            #loss = loss - self.pos_weight * pos
            #if self.crisp_weight != 0:
            #    loss = loss + self.crisp_weight * crisp
            if self.diff_weight != 0:
                loss = loss - self.diff_weight * diff
            model.zero_reg()

            optim.zero_grad(set_to_none=True)
            loss.backward()
            optim.step()
            
        return step
        
    def new_model(self):
        return FuzzyMNIST(
            logic=self.flogic, 
            layers=self.layers,
            clauses_per_output=self.clauses_per_output,
            signed_clauses=False,
            maxsat=self.maxsat
        ).to(device)

In [ ]:
def defer_regime(*args, **kargs):
    def return_regime():
        return MNISTRealRegime(*args, **kargs)
    return return_regime

sslogic = SchweizerSklarLogic(torch.tensor(-4.0).to(device))
layers = [(28*28, 128), (128, 64, 10)]

regimes = [
    defer_regime(logic=sslogic, layers=layers, clauses_per_output=4, diff_weight=1e-5, maxsat=True, lr=1e-3),
    defer_regime(logic=sslogic, layers=layers, clauses_per_output=4, diff_weight=1e-3, maxsat=True, lr=1e-3),
    defer_regime(logic=sslogic, layers=layers, clauses_per_output=4, diff_weight=1, maxsat=True, lr=1e-3),
]

torch.autograd.set_detect_anomaly(True)
if True:
    regs = [regime_f() for regime_f in regimes]
    for reg in regs:
        reg.load_latest_models()
        reg.load_all_results()
    
    for i in range(0, 100):
        for reg in regs:
            #try:
                reg.loop_until(1, 1*(i+1))
            #except:
            #    pass

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1e-05 | MAXSAT | 4 CLAUSES…

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1e-05 | MAXSAT | 4 CLAUSES…

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1e-05 | MAXSAT | 4 CLAUSES…

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1e-05 | MAXSAT | 4 CLAUSES…

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1e-05 | MAXSAT | 4 CLAUSES…

...

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1 | MAXSAT | 4 CLAUSES] Ru…

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1 | MAXSAT | 4 CLAUSES] Ru…

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1 | MAXSAT | 4 CLAUSES] Ru…

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1 | MAXSAT | 4 CLAUSES] Ru…

[MNIST | ADAM, LR = 0.001 | SS LOGIC | LAYERS = (784,128) (128,64,10) | DIFF WEIGHT 1e-05 | MAXSAT | 4 CLAUSES…

In [ ]:
0 ** 1

In [ ]:
reg = regimes[0]()
reg.load_latest_models()
reg.load_all_results()
model = reg.models[0].to(device)

In [ ]:
reg.results[0]

In [ ]:
feature = nn.Parameter(torch.rand((1, 28, 28)), requires_grad=True)
optim = torch.optim.Adam((feature,), lr=1e-2)

In [ ]:
torch.autograd.set_detect_anomaly(True)
for i in tqdm(range(5000)):
    bounded = feature.sigmoid().flatten(start_dim=1).to(device)
    a_0 = model.layers[0](bounded)
    c_1 = torch.zeros_like(a_0, device=a_0.device)
    c_1[:,0] = -1
    a_1 = a_0 * c_1
    loss = a_1.sum()
    
    if i % 500 == 0:
        print(loss)
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    
plt.imshow(feature.sigmoid().squeeze().detach().cpu().numpy())

In [ ]:
values = model.layers[1].fuzzy_m.value()
torch.min(values, 1 - values).mean()

In [ ]:
from matplotlib import cm

for imgs, labels in tqdm(reg.training_dataloader(1)):
    i = 7
    img = imgs[i:i+1,:]
    label = labels[i]
    img = nn.Parameter(imgs[i:i+1,:], requires_grad=True)
    a_0 = model.layers[0](img.flatten(start_dim=1).to(device))
    a_1 = model.layers[1](a_0)
    loss = -a_0[:,45].sum()
    img.grad = None
    loss.backward()
    fig, ax = plt.subplots(3)
    ax[0].imshow(-img.grad.reshape(28,28).detach().cpu().numpy(), cmap="bwr")
    ax[1].imshow(img.reshape(28,28).detach().cpu().numpy())
    print(a_1)
    #print(model.layers[1].fuzzy_m.value()[2,:,:] > 0.5)
    #print(a_0 > 0.9)
    features = torch.logical_and(model.layers[1].fuzzy_m.value()[label,0,:] > 0.5, a_0 > 0.9)
    print(features)
    
    new_img = nn.Parameter(torch.rand((1, 28, 28)), requires_grad=True)
    optim = torch.optim.Adam((new_img,), lr=1e-2)
    
    for i in tqdm(range(5000)):
        bounded = new_img.sigmoid().flatten(start_dim=1).to(device)
        a_0 = model.layers[0](bounded)
        c_1 = torch.zeros_like(a_0, device=a_0.device)
        c_1[features] = -1
        a_1 = a_0 * c_1
        loss = a_1.sum()

        if i % 500 == 0:
            print(loss)

        optim.zero_grad()
        loss.backward()
        optim.step()
    
    ax[2].imshow(new_img.sigmoid().squeeze().detach().cpu().numpy())
    
    break

In [ ]:
model(new_img.sigmoid().detach().to(device))

# MNIST Auto Encoding

In [ ]:
class MNISTAutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 4),
            nn.Sigmoid(),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(4, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 28*28),
            nn.LeakyReLU(),
        )
        
    def encode(self, input):
        return self.encoder(input)
    
    def decode(self, latent):
        decoded = self.decoder(latent)
        return decoded.reshape(*decoded.shape[:-1], 1, 28, 28)
    
    def forward(self, input):
        return self.decode(self.encode(input))

In [ ]:
ae_model = MNISTAutoEncoder().to(device)
no_ae_epochs = 30

In [ ]:
def train_ae():
    optim = torch.optim.Adam(ae_model.parameters(), lr=1e-3)

    losses = []
    for epoch_no in range(1, no_ae_epochs + 1):
        for imgs, _ in tqdm(trainloader, desc="Training Epoch %s" % epoch_no, leave=False):
            decoding = ae_model(imgs.to(device))
            loss = ((imgs.to(device) - decoding) ** 2).sum()
            losses.append(loss.item())

            optim.zero_grad()
            loss.backward()
            optim.step()
            
    return torch.tensor(losses)

train_ae()

In [ ]:
for imgs, _ in testloader:
    i=14
    img = imgs[i:i+1]
    decoding = ae_model(img.to(device))
    fig, ax = plt.subplots(2)
    ax[0].imshow(img.squeeze().cpu().numpy())
    ax[1].imshow(decoding.squeeze().detach().cpu().numpy())
    break

In [ ]:
from matplotlib import cm

for imgs, labels in tqdm(reg.testing_dataloader(1)):
    i = 15
    img = imgs[i:i+1,:]
    label = labels[i]
    img = nn.Parameter(imgs[i:i+1,:], requires_grad=True)
    a_0 = model.layers[0](img.flatten(start_dim=1).to(device))
    a_1 = model.layers[1](a_0)
    loss = -a_0[:,45].sum()
    img.grad = None
    loss.backward()
    fig, ax = plt.subplots(3)
    ax[0].imshow(-img.grad.reshape(28,28).detach().cpu().numpy(), cmap="bwr")
    ax[1].imshow(img.reshape(28,28).detach().cpu().numpy())
    print(a_1)
    #print(model.layers[1].fuzzy_m.value()[2,:,:] > 0.5)
    #print(a_0 > 0.9)
    features = (model.layers[1].fuzzy_m.value()[label,0,:]).unsqueeze(0)#torch.logical_and(model.layers[1].fuzzy_m.value()[label,0,:] > 0.5, a_0 > 0.95)
    print(features > 0.5)
    
    new_img = FuzzyParam((1, 4)).to(device)
    optim = torch.optim.Adam(new_img.parameters(), lr=1e-2)
    
    for i in tqdm(range(1500)):
        input = ae_model.decode(new_img.value())
        a_0 = model.layers[0](input.flatten(start_dim=1))
        c_1 = torch.zeros_like(a_0, device=a_0.device)
        c_1[features > 0.5] = -1
        a_1 = c_1 * a_0
        #a_1 = -model.logic.conjoin(model.logic.implies(features.detach(), a_0), dim=-1)
        loss = a_1.sum()

        if i % 500 == 0:
            print(loss)

        optim.zero_grad()
        loss.backward()
        optim.step()
    
    ax[2].imshow(ae_model.decode(new_img.value()).squeeze().detach().cpu().numpy())
    
    break

In [ ]:
model(ae_model.decode(new_img.value()))